In [6]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from pathlib import Path
import glob


import gtsam
from gtsam import Pose3, Rot3, Point3
from gtsam.symbol_shorthand import X

## Load KITTI Data

Load all the KITTI data

In [8]:
def load_kitti_scan(path):
    points = np.fromfile(path, dtype=np.float32)
    return points.reshape(-1, 4)


kitti_root = Path("/mnt/storage/datasets/kitti/2011_09_30/2011_09_30_drive_0027_sync/velodyne_points")
velodyne_dir = kitti_root / "data"


scan_files = sorted(glob.glob(f"{velodyne_dir}/**"))
print(scan_files)
scan0 = load_kitti_scan(scan_files[0])
scan0.shape

['/mnt/storage/datasets/kitti/2011_09_30/2011_09_30_drive_0027_sync/velodyne_points/data/0000000000.bin', '/mnt/storage/datasets/kitti/2011_09_30/2011_09_30_drive_0027_sync/velodyne_points/data/0000000001.bin', '/mnt/storage/datasets/kitti/2011_09_30/2011_09_30_drive_0027_sync/velodyne_points/data/0000000002.bin', '/mnt/storage/datasets/kitti/2011_09_30/2011_09_30_drive_0027_sync/velodyne_points/data/0000000003.bin', '/mnt/storage/datasets/kitti/2011_09_30/2011_09_30_drive_0027_sync/velodyne_points/data/0000000004.bin', '/mnt/storage/datasets/kitti/2011_09_30/2011_09_30_drive_0027_sync/velodyne_points/data/0000000005.bin', '/mnt/storage/datasets/kitti/2011_09_30/2011_09_30_drive_0027_sync/velodyne_points/data/0000000006.bin', '/mnt/storage/datasets/kitti/2011_09_30/2011_09_30_drive_0027_sync/velodyne_points/data/0000000007.bin', '/mnt/storage/datasets/kitti/2011_09_30/2011_09_30_drive_0027_sync/velodyne_points/data/0000000008.bin', '/mnt/storage/datasets/kitti/2011_09_30/2011_09_30_dri

(122626, 4)

# Process points

Import the points from the scan, then downsample and estimates the normals.

In [9]:
def preprocess(points, voxel=0.2):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points[:, :3])
    
    
    pcd = pcd.voxel_down_sample(voxel)
    pcd.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=1.0, max_nn=30)
    )
    return pcd


pcd0 = preprocess(scan0)

In [11]:
def compute_curvature(pcd, k=20):
    pts = np.asarray(pcd.points)
    tree = o3d.geometry.KDTreeFlann(pcd)

    curvatures = []
    for i in range(len(pts)):
        _, idx, _ = tree.search_knn_vector_3d(pts[i], k)
        neighbors = pts[idx]
        cov = np.cov(neighbors.T)
        eigvals = np.linalg.eigvalsh(cov)
        curvature = eigvals[0] / eigvals.sum()
        curvatures.append(curvature)
    
    return np.array(curvatures)


curv = compute_curvature(pcd0)

In [12]:
def extract_features(pcd, curv, edge_th=0.05, plane_th=0.01):
    pts = np.asarray(pcd.points)
    edges = pts[curv > edge_th]
    planes = pts[curv < plane_th]
    return edges, planes


edge_pts, plane_pts = extract_features(pcd0, curv)

## Scan to Scan Odometry


In [13]:
def estimate_odometry(pcd_src, pcd_tgt, init=np.eye(4)):
    result = o3d.pipelines.registration.registration_icp(
    pcd_src, pcd_tgt,
    max_correspondence_distance=1.0,
    init=init,
    estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPlane()
    )
    return result.transformation

In [14]:
pcd1 = preprocess(load_kitti_scan(scan_files[1]))
T_01 = estimate_odometry(pcd0, pcd1)
T_01

array([[ 9.99979250e-01,  6.43901080e-03, -1.98312614e-04,
        -9.44760222e-02],
       [-6.43908427e-03,  9.99979200e-01, -3.72082496e-04,
        -7.83089403e-03],
       [ 1.95912646e-04,  3.73351727e-04,  9.99999911e-01,
        -8.78445276e-05],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [ ]:
# estimate odometry between every pairwise pair of scans, starting with 